In [1]:
# Python3 program for the above approach
#https://www.geeksforgeeks.org/m-coloring-problem/
# Number of vertices in the graph
# define 4 4

# check if the colored
# graph is safe or not


def isSafe(graph, color):

	# check for every edge
	for i in range(4):
		for j in range(i + 1, 4):
			if (graph[i][j] and color[j] == color[i]):
				return False
	return True

# /* This function solves the m Coloring
# problem using recursion. It returns
# false if the m colours cannot be assigned,
# otherwise, return true and prints
# assignments of colours to all vertices.
# Please note that there may be more than
# one solutions, this function prints one
# of the feasible solutions.*/


def graphColoring(graph, m, i, color):

	# if current index reached end
	if (i == 4):

		# if coloring is safe
		if (isSafe(graph, color)):

			# Print the solution
			printSolution(color)
			return True
		return False

	# Assign each color from 1 to m
	for j in range(1, m + 1):
		color[i] = j

		# Recur of the rest vertices
		if (graphColoring(graph, m, i + 1, color)):
			return True
		color[i] = 0
	return False

# /* A utility function to print solution */


def printSolution(color):
	print("Solution Exists:" " Following are the assigned colors ")
	for i in range(4):
		print(color[i], end=" ")


# Driver code
if __name__ == '__main__':

	# /* Create following graph and
	# test whether it is 3 colorable
	# (3)---(2)
	# | / |
	# | / |
	# | / |
	# (0)---(1)
	# */
	graph = [
		[0, 1, 1, 1],
		[1, 0, 1, 0],
		[1, 1, 0, 1],
		[1, 0, 1, 0],
	]
	m = 3 # Number of colors

	# Initialize all color values as 0.
	# This initialization is needed
	# correct functioning of isSafe()
	color = [0 for i in range(4)]

	# Function call
	if (not graphColoring(graph, m, 0, color)):
		print("Solution does not exist")

# This code is contributed by mohit kumar 29


Solution Exists: Following are the assigned colors 
1 2 3 2 

In [ ]:
pip install dimod


In [ ]:
pip install dwave-system

In [30]:
#https://docs.ocean.dwavesys.com/en/stable/examples/map_coloring.html
import networkx as nx
import matplotlib.pyplot as plt    
from dimod.generators import combinations
from dimod import BinaryQuadraticModel, ExactSolver
from dwave.system import DWaveSampler, EmbeddingComposite

In [31]:
provinces = ['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'NU', 'ON', 'PE',
             'QC', 'SK', 'YT']
neighbors = [('AB', 'BC'), ('AB', 'NT'), ('AB', 'SK'), ('BC', 'NT'), ('BC', 'YT'),
             ('MB', 'NU'), ('MB', 'ON'), ('MB', 'SK'), ('NB', 'NS'), ('NB', 'QC'),
             ('NL', 'QC'), ('NT', 'NU'), ('NT', 'SK'), ('NT', 'YT'), ('ON', 'QC')]

In [32]:
colors = ['y', 'g', 'r', 'b']

In [ ]:
#In the code below, bqm_one_color uses a one-hot penalty model to formulate the constraint that each node (province) select a single color.

bqm_one_hot = combinations(['a', 'b'], 1)
print(bqm_one_hot)
BinaryQuadraticModel({'a': -1.0, 'b': -1.0}, {('b', 'a'): 2.0}, 1.0, 'BINARY')
print(ExactSolver().sample(bqm_one_hot))

In [34]:
#Set a one-hot constraint on the four binary variables representing the possible colors for each province.

bqm_one_color = BinaryQuadraticModel('BINARY')
for province in provinces:
  variables = [province + "_" + c for c in colors]
  bqm_one_color.update(combinations(variables, 1))

In [ ]:
#As in the illustrative example above, the binary variables created for each province set linear biases of -1 and 
#quadratic biases of 2 that penalize states where more than a single color is selected.

print([variable for variable in bqm_one_color.variables if provinces[0] in variable])

print(bqm_one_color.linear['AB_y'], bqm_one_color.quadratic['AB_y', 'AB_g'])

In [ ]:
# In the code below, bqm_neighbors represents the constraint that two nodes (provinces) with a shared edge (border) not both select the same color.

bqm_and = BinaryQuadraticModel({}, {'ab': -1}, 1, 'BINARY')
print(ExactSolver().sample(bqm_and))

In [ ]:
bqm_and_plus = BinaryQuadraticModel({}, {'ab': 1}, 0, 'BINARY')
print(ExactSolver().sample(bqm_and_plus))

In [39]:
bqm_neighbors  = BinaryQuadraticModel('BINARY')
for neighbor in neighbors:
  v, u = neighbor
  interactions = [(v + "_" + c, u + "_" + c) for c in colors]
  for interaction in interactions:
     bqm_neighbors.add_quadratic(interaction[0], interaction[1], 1)

In [ ]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=1000,label='SDK Examples - Map Coloring BQM') 
best = sampleset.first     

In [ ]:
if best.energy > 0:        
    print("Failed to color map. Try sampling again.")

In [26]:
def plot_map(sample):
    G = nx.Graph()
    G.add_nodes_from(provinces)
    G.add_edges_from(neighbors)
    # Create a {province: selected color} dict
    color_map = {}
    for province in provinces:
      for c in colors:
       if sample[province + '_' + c]:
           color_map[province] = c
    # Plot with the selected colors
    node_colors = [color_map.get(node) for node in G.nodes()]
    nx.draw_circular(G, with_labels=True, node_color=node_colors, node_size=3000, cmap=plt.cm.rainbow)
    plt.show()

In [ ]:
plot_map(best.sample)  